In [2]:
import numpy as np
import random
import syft as sy
import json
from syft.core.frameworks.numpy import NumpyEncoder

In [16]:
x = array([1,2,3,4])
me = sy.local_worker
hook = me.hook

In [17]:
bob = sy.VirtualWorker(hook=hook)

In [18]:
x = array([1,2,3,4])

In [19]:
xptr = x.send(bob)

In [23]:
xptr

array_ptr(['data is remote'], dtype='<U14')

In [20]:
x = xptr.get()

In [21]:
x

array([1, 2, 3, 4])